In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import sys
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter



2023-07-06 22:52:12.957583: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:

criterion = nn.CrossEntropyLoss()

# usage of criterion
# loss = criterion(output, target)

# example 
input = torch.randn(3, 5, requires_grad=True)

target = torch.empty(3, dtype=torch.long).random_(5)

print(input, input.shape)
print(target, target.shape)

loss = criterion(input, target)
print(loss)



tensor([[ 0.0550,  1.1971,  0.9745,  1.5122,  0.5402],
        [-0.6484,  0.8938,  1.1234,  0.8191,  1.0692],
        [ 1.2203,  1.0927, -0.7158,  1.1697,  1.1498]], requires_grad=True) torch.Size([3, 5])
tensor([4, 0, 0]) torch.Size([3])
tensor(2.1582, grad_fn=<NllLossBackward0>)


In [14]:
import torch
import torch.nn as nn

# Assuming your output and target tensors are already defined
output = torch.randn(32, 1024, 2)
target = torch.randint(0, 2, (32, 1024))  # Random target tensor for demonstration

# Define the CrossEntropyLoss criterion
criterion = nn.CrossEntropyLoss()

print(output.shape)
print(target.shape)


# Calculate the loss
loss = criterion(output.view(-1, 2), target.view(-1))

print(output.view(-1, 2).shape)
print(target.view(-1).shape)
print(loss)



torch.Size([32, 1024, 2])
torch.Size([32, 1024])
torch.Size([32768, 2])
torch.Size([32768])
tensor(0.9010)


In [7]:
from simple_model import PointNetSegmentation
import torch 
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

#read data
data = np.load('point_cloud_data_test.npz')

# separate point cloud data into points and labels
points = data['points']
labels = data['labels']

# visualize point cloud
import open3d as o3d

# Create Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

# Assign colors based on labels
label_to_color = {1: [1, 0, 0], 0: [0, 1, 0]}
colors = [label_to_color[label] for label in labels]
pcd.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])



# convert numpy array to tensor
points = torch.from_numpy(points)
labels = torch.from_numpy(labels)

print(points.shape)
print(labels.shape)

# reshape points to (1, 1024, 3)
points = points.reshape(1, 1024, 3)
print(points.shape)



torch.Size([1024, 3])
torch.Size([1024])
torch.Size([1, 1024, 3])


In [8]:
model = PointNetSegmentation(num_classes=2)

# load model
model.load_state_dict(torch.load('model.pth'))
model.to(device)

# set model to evaluation mode
model.eval()

point_clouds_2 = points.transpose(2, 1).to(device).float()  # (batch_size, 3, num_points)
print(point_clouds_2.shape)
labels = labels.to(device)  # (batch_size, num_points)

# Forward pass
outputs = model(point_clouds_2)  # (batch_size, num_points, num_classes)
predicted_labels = torch.argmax(outputs, dim=2)  # (batch_size, num_points)

print(outputs.shape)
print(predicted_labels.shape)

# Convert tensors to numpy arrays
points = points.cpu().numpy()
predicted_labels = predicted_labels.cpu().numpy()


torch.Size([1, 3, 1024])
torch.Size([1, 1024, 2])
torch.Size([1, 1024])


In [9]:
import open3d

# convert points to shape (1024, 3)
points = points.reshape(1024, 3)
print(points.shape)

# convert predicted_labels to shape (1024,)
predicted_labels = predicted_labels.reshape(1024,)
print(predicted_labels.shape)

# create point cloud with points
point_cloud = open3d.geometry.PointCloud()
point_cloud.points = open3d.utility.Vector3dVector(points)

# assign colors to point cloud based on predicted labels
color_palette = np.array([[0, 255, 0],  # Class 0 color (red)
                            [255, 0, 0]])  # Class 1 color (blue)

point_colors = color_palette[predicted_labels.reshape(-1)]

# Assign colors to point cloud
point_cloud.colors = open3d.utility.Vector3dVector(point_colors / 255.0)

# Visualize the point cloud
open3d.visualization.draw_geometries([point_cloud])




(1024, 3)
(1024,)
